In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # 启用CUDA阻塞调试
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # 避免内存碎片
import torch

from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from modelscope import snapshot_download
from modelscope import Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor, Qwen2_5OmniThinkerForConditionalGeneration

from transformers.models.qwen2_vl.video_processing_qwen2_vl import Qwen2VLVideoProcessor
from transformers.video_utils import VideoMetadata
from typing import Optional
from qwen_omni_utils import process_mm_info
import json
from torch.utils.data import Dataset
from transformers.image_utils import SizeDict

class OmniVideoConversationDataset(Dataset):
    def __init__(
        self,
        json_path: str,
        video_root: str,
    ):
        with open(json_path, "r") as f:
            self.data = json.load(f)

        self.video_root = video_root

    def __len__(self):
        return len(self.data)

    def _build_text(self, conversations):
        messages = []
        for turn in conversations:
            if turn["from"] == "human":
                role = "user"
            elif turn["from"] == "gpt":
                role = "assistant"
            else:
                continue

            messages.append({
                "role": role,
                "content": turn["value"]
            })

        return messages

    def __getitem__(self, idx):
        sample = self.data[idx]
        video_id = sample["id"]
        video_path = os.path.join(self.video_root, f"{video_id}.mp4")

        conversation = [
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": video_path},
                    {"type": "text", "text": sample["conversations"][0]["value"]},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": sample["conversations"][1]["value"]},
                ],
            },
        ]

        return {"conversation": conversation}

class QwenOmniDataCollator:
    def __init__(self, processor):
        self.processor = processor
        self.tokenizer = processor.tokenizer

    def __call__(self, features):
        texts = []
        videos = []
        audios = []
        labels_list = []

        for f in features:
            conversation = f["conversation"]

            # ---------- 1. 拼完整 prompt ----------
            full_text = self.processor.apply_chat_template(
                conversation,
                tokenize=False,
                add_generation_prompt=False,
            )

            # ---------- 2. 构造 labels（后缀 assistant） ----------
            assistant_text = conversation[-1]["content"][0]["text"]

            full_ids = self.tokenizer(
                full_text,
                add_special_tokens=False,
            )["input_ids"]

            assistant_ids = self.tokenizer(
                assistant_text,
                add_special_tokens=False,
            )["input_ids"]

            labels = [-100] * len(full_ids)
            labels[-len(assistant_ids):] = assistant_ids

            texts.append(full_text)
            labels_list.append(labels)

            # ---------- 3. 多模态 ----------
            for msg in conversation:
                if msg["role"] == "user":
                    for ele in msg["content"]:
                        if ele.get("type") == "video":
                            ele["fps"] = 0.5
                            ele["max_frames"] = 50
                            # ele["min_pixels"] = 64 * 28 * 28
                            # ele["max_pixels"] = 64 * 28 * 28

            audios_, _, videos_ = process_mm_info(
                conversation, use_audio_in_video=True
            )

            videos.append(videos_[0] if videos_ else None)
            audios.append(audios_[0] if audios_ else None)

        # ---------- 4. 一次性 processor ----------
        batch = self.processor(
            text=texts,
            videos=videos,
            audio=audios,
            padding=True,
            return_tensors="pt",
            use_audio_in_video=True,
        )

        print(batch["video_grid_thw"].shape) 
        print(batch["pixel_values_videos"].shape) 
        for k, v in batch.items(): 
            if isinstance(v, torch.Tensor): 
                print(k, v.shape, v.numel() * v.element_size() / 1024**3, "GB")


        # ---------- 5. pad labels ----------
        max_len = batch["input_ids"].shape[1]
        padded_labels = []

        for lab in labels_list:
            padded = lab + [-100] * (max_len - len(lab))
            padded_labels.append(padded)

        batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)

        return batch


train_dataset = OmniVideoConversationDataset(
    json_path="../../LongVALE/data/longvale-sft-bp-7k.json",
    video_root="../../LongVALE/raw_videos_train/video_train_7240/"
)


model_path = snapshot_download(
    'Qwen/Qwen2.5-Omni-3B',
    cache_dir="../../Qwen/cache/modelscope"
)

model = Qwen2_5OmniThinkerForConditionalGeneration.from_pretrained(
    model_path,
    dtype=torch.bfloat16,
    device_map="cuda:0",
    trust_remote_code=True,
    use_safetensors=True
)

class FixedResQwen2VLVideoProcessor(Qwen2VLVideoProcessor):
    def _preprocess(
        self, videos, do_resize=True, size=None, interpolation=None, **kwargs
    ):
        # 固定分辨率
        fixed_size = SizeDict(height=224, width=224)
        for i, video in enumerate(videos):
            videos[i] = self.resize(video, size=fixed_size, interpolation=interpolation)
        return super()._preprocess(videos, do_resize=False, size=fixed_size, interpolation=interpolation, **kwargs)
    
video_processor = FixedResQwen2VLVideoProcessor.from_pretrained(model_path)

processor = Qwen2_5OmniProcessor.from_pretrained(
    model_path,
    video_processor=video_processor,
)


# 配置LoRA
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    # task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


for name, param in model.named_parameters():
    if (
        "audio_tower" in name
        or "visual" in name
    ):
        param.requires_grad = False

model.gradient_checkpointing_enable()
model.config.use_cache = False

model.print_trainable_parameters()

# 检查模型是否在训练模式
model.train()
print(f"Model is in training mode: {model.training}")

batch_size = 1

args = TrainingArguments(
    output_dir="./r_models",
    remove_unused_columns=False,
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=2,
    bf16=True,
    fp16=False,
    num_train_epochs=2,
    logging_steps=5,
    load_best_model_at_end=False,
)

data_collator = QwenOmniDataCollator(processor)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)


# print("\n=== 进行前向传播测试 ===")
# with torch.set_grad_enabled(True):

#     sample = next(iter(train_dataset))
#     batch = data_collator([sample])
    

#     device = model.device
#     batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
    
#     # 前向传播
#     outputs = model(**batch)
#     loss = outputs.loss
    
#     print(f"Loss: {loss}")
#     print(f"Loss requires_grad: {loss.requires_grad}")
    
#     # 反向传播
#     if loss.requires_grad:
#         loss.backward()
#         print("反向传播成功")
        
#         # 检查梯度
#         for name, param in model.named_parameters():
#             if param.requires_grad and param.grad is not None:
#                 print(f"参数 '{name}' 有梯度")
#                 break
#         else:
#             print("警告：没有发现任何参数的梯度")
#     else:
#         print("错误：loss没有requires_grad属性")

2025-12-19 00:55:38,459 - modelscope - INFO - Target directory already exists, skipping creation.
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
The model is already on multiple devices. Skipping the move to device specified in `args`.


trainable params: 22,413,312 || all params: 4,734,622,720 || trainable%: 0.4734
Model is in training mode: True


In [ ]:
print("\n=== 开始训练 ===")
trainer.train()


=== 开始训练 ===


/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
qwen-vl-utils using decord to read video.


torch.Size([1, 3])
torch.Size([6400, 1176])
input_ids torch.Size([1, 9210]) 6.861984729766846e-05 GB
attention_mask torch.Size([1, 9210]) 6.861984729766846e-05 GB
pixel_values_videos torch.Size([6400, 1176]) 0.02803802490234375 GB
video_grid_thw torch.Size([1, 3]) 2.2351741790771484e-08 GB
video_second_per_grid torch.Size([1]) 3.725290298461914e-09 GB
feature_attention_mask torch.Size([1, 30000]) 0.00011175870895385742 GB
input_features torch.Size([1, 128, 30000]) 0.01430511474609375 GB


/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


torch.Size([1, 3])
torch.Size([2560, 1176])
input_ids torch.Size([1, 1807]) 1.3463199138641357e-05 GB
attention_mask torch.Size([1, 1807]) 1.3463199138641357e-05 GB
pixel_values_videos torch.Size([2560, 1176]) 0.0112152099609375 GB
video_grid_thw torch.Size([1, 3]) 2.2351741790771484e-08 GB
video_second_per_grid torch.Size([1]) 3.725290298461914e-09 GB
feature_attention_mask torch.Size([1, 30000]) 0.00011175870895385742 GB
input_features torch.Size([1, 128, 30000]) 0.01430511474609375 GB


/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


torch.Size([1, 3])
torch.Size([6400, 1176])
input_ids torch.Size([1, 5538]) 4.126131534576416e-05 GB
attention_mask torch.Size([1, 5538]) 4.126131534576416e-05 GB
pixel_values_videos torch.Size([6400, 1176]) 0.02803802490234375 GB
video_grid_thw torch.Size([1, 3]) 2.2351741790771484e-08 GB
video_second_per_grid torch.Size([1]) 3.725290298461914e-09 GB
feature_attention_mask torch.Size([1, 30000]) 0.00011175870895385742 GB
input_features torch.Size([1, 128, 30000]) 0.01430511474609375 GB


/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
from inspect import signature
print(signature(Qwen2_5OmniForConditionalGeneration.forward))
print(signature(Qwen2_5OmniThinkerForConditionalGeneration.forward))

(self, *input: Any) -> None
(self, input_ids: Optional[torch.LongTensor] = None, input_features: Optional[torch.FloatTensor] = None, pixel_values: Optional[torch.FloatTensor] = None, pixel_values_videos: Optional[torch.FloatTensor] = None, image_grid_thw: Optional[torch.LongTensor] = None, video_grid_thw: Optional[torch.LongTensor] = None, attention_mask: Optional[torch.Tensor] = None, feature_attention_mask: Optional[torch.Tensor] = None, audio_feature_lengths: Optional[torch.LongTensor] = None, position_ids: Optional[torch.LongTensor] = None, past_key_values: Optional[transformers.cache_utils.Cache] = None, inputs_embeds: Optional[torch.FloatTensor] = None, rope_deltas: Optional[torch.LongTensor] = None, labels: Optional[torch.LongTensor] = None, use_cache: Optional[bool] = None, output_attentions: Optional[bool] = None, output_hidden_states: Optional[bool] = None, return_dict: Optional[bool] = None, use_audio_in_video: Optional[bool] = None, cache_position: Optional[torch.LongTensor

In [3]:
def verify_lora_gradients(model):
    """验证LoRA梯度流"""
    model.train()
    
    # 创建测试数据
    test_inputs = {
        'input_ids': torch.randint(0, 1000, (1, 10)).cuda(),
        'attention_mask': torch.ones(1, 10).cuda(),
        'labels': torch.randint(0, 1000, (1, 10)).cuda(),
    }
    
    # 前向传播
    outputs = model(**test_inputs)
    loss = outputs.loss
    
    print(f"Loss: {loss.item():.4f}")
    print(f"Loss requires_grad: {loss.requires_grad}")
    
    if loss.requires_grad:
        # 反向传播
        loss.backward()
        
        # 检查梯度
        gradients_found = False
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                gradients_found = True
                grad_norm = param.grad.norm().item()
                if 'lora' in name:
                    print(f"  ✓ LoRA梯度: {name} | norm={grad_norm:.6f}")
        
        if not gradients_found:
            print("  ⚠ 没有找到梯度")
    else:
        print("  ✗ Loss没有requires_grad")

verify_lora_gradients(model)

Loss: 11.3224
Loss requires_grad: True
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight | norm=0.096089
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight | norm=0.093929
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight | norm=1.119280
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight | norm=0.544598
  ✓ LoRA梯度: base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weig

In [3]:
# 取训练集的第一个样本
sample = train_dataset[0]
batch = data_collator([sample])

# 将数据放到 GPU
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        batch[k] = v.cuda()

# 调用模型
outputs = model(
    input_ids=batch["input_ids"],
    attention_mask=batch["attention_mask"],
    labels=batch["labels"],
)

# 输出 loss
print("loss:", outputs.loss)


/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
qwen-vl-utils using decord to read video.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


torch.Size([1, 3])
torch.Size([6400, 1176])
input_ids torch.Size([1, 8149]) 6.0714781284332275e-05 GB
attention_mask torch.Size([1, 8149]) 6.0714781284332275e-05 GB
pixel_values_videos torch.Size([6400, 1176]) 0.02803802490234375 GB
video_grid_thw torch.Size([1, 3]) 2.2351741790771484e-08 GB
video_second_per_grid torch.Size([1]) 3.725290298461914e-09 GB
feature_attention_mask torch.Size([1, 30000]) 0.00011175870895385742 GB
input_features torch.Size([1, 128, 30000]) 0.01430511474609375 GB


/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


loss: tensor(13.5661, device='cuda:0')


In [4]:
labels = batch["labels"]      # [batch_size, seq_len]
input_ids = batch["input_ids"] # [batch_size, seq_len]
tokenizer = processor.tokenizer

batch_size = labels.shape[0]

for i in range(batch_size):
    print(f"=== Sample {i} ===")
    effective_tokens = []
    effective_texts = []
    
    for j in range(labels.shape[1]):
        label = labels[i, j].item()
        token_id = input_ids[i, j].item()
        if label != -100:  # 有效 token
            effective_tokens.append(token_id)
            effective_texts.append(tokenizer.decode([token_id]))
    
    print("有效 token 数量:", len(effective_tokens))
    print("有效 token id:", effective_tokens)
    print("对应文本:", effective_texts)
    print("完整文本:", tokenizer.decode(effective_tokens))


=== Sample 0 ===
有效 token 数量: 10
有效 token id: [151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656]
对应文本: ['<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>']
完整文本: <|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|>


In [10]:
# 测试有效 token 对应文本
import torch

# 取训练集第一个样本
sample = train_dataset[0]

# 通过 data_collator 生成 batch
batch = data_collator([sample])

labels = batch["labels"]       # [batch_size, seq_len]
input_ids = batch["input_ids"] # [batch_size, seq_len]
tokenizer = processor.tokenizer

batch_size = labels.shape[0]

for i in range(batch_size):
    print(f"=== Sample {i} ===")
    
    total_tokens = labels.shape[1]
    effective_tokens = []
    effective_texts = []
    
    for j in range(total_tokens):
        label = labels[i, j].item()
        token_id = input_ids[i, j].item()
        if label != -100:  # 有效 token
            effective_tokens.append(token_id)
            effective_texts.append(tokenizer.decode([token_id]))
    
    print("总 token 数量:", total_tokens)
    print("有效 token 数量:", len(effective_tokens))
    print("有效 token id:", effective_tokens)
    print("对应文本:", effective_texts)
    print("有效 token 拼接文本:", tokenizer.decode(effective_tokens))


/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


torch.Size([1, 3])
torch.Size([6400, 1176])
input_ids torch.Size([1, 8149]) 6.0714781284332275e-05 GB
attention_mask torch.Size([1, 8149]) 6.0714781284332275e-05 GB
pixel_values_videos torch.Size([6400, 1176]) 0.02803802490234375 GB
video_grid_thw torch.Size([1, 3]) 2.2351741790771484e-08 GB
video_second_per_grid torch.Size([1]) 3.725290298461914e-09 GB
feature_attention_mask torch.Size([1, 30000]) 0.00011175870895385742 GB
input_features torch.Size([1, 128, 30000]) 0.01430511474609375 GB
=== Sample 0 ===
总 token 数量: 8149
有效 token 数量: 10
有效 token id: [151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656, 151656]
对应文本: ['<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>', '<|VIDEO|>']
有效 token 拼接文本: <|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|>


In [12]:
# 取训练集第一个样本
sample = train_dataset[0]

# 通过 data_collator 生成 batch
batch = data_collator([sample])

labels = batch["labels"]       # [batch_size, seq_len]
input_ids = batch["input_ids"] # [batch_size, seq_len]
tokenizer = processor.tokenizer

for i in range(labels.shape[0]):
    effective_ids = []
    ignored_ids = []
    
    for j in range(labels.shape[1]):
        token_id = input_ids[i, j].item()
        label = labels[i, j].item()
        if label != -100:
            effective_ids.append(token_id)
        else:
            ignored_ids.append(token_id)
    
    effective_text = tokenizer.decode(effective_ids)
    ignored_text = tokenizer.decode(ignored_ids)
    
    print(f"=== Sample {i} ===")
    print("有效 token 拼接文本:")
    print(effective_text)
    print("无效 token 拼接文本:")
    print(ignored_text)



/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


torch.Size([1, 3])
torch.Size([6400, 1176])
input_ids torch.Size([1, 8149]) 6.0714781284332275e-05 GB
attention_mask torch.Size([1, 8149]) 6.0714781284332275e-05 GB
pixel_values_videos torch.Size([6400, 1176]) 0.02803802490234375 GB
video_grid_thw torch.Size([1, 3]) 2.2351741790771484e-08 GB
video_second_per_grid torch.Size([1]) 3.725290298461914e-09 GB
feature_attention_mask torch.Size([1, 30000]) 0.00011175870895385742 GB
input_features torch.Size([1, 128, 30000]) 0.01430511474609375 GB
=== Sample 0 ===
有效 token 拼接文本:
<|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|>
无效 token 拼接文本:
<|im_start|>system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.<|im_end|>
<|im_start|>user
<|vision_bos|><|audio_bos|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO|><|VIDEO